## Selecting Rows with Core or ORM

For both Core and ORM, the `select()` function generates a `Select construct` which is used for all `SELECT` queries. Passed to methods like _`Connection.execute()` in Core_ and _`Session.execute()` in ORM_, a `SELECT statement` is emitted in the _current transaction_ and the result rows available via the returned `Result` object.

#### Initialize tables with some data

In [1]:
from sqlalchemy import (
    MetaData, Table, Column, Integer, String, ForeignKey, JSON, create_engine, insert,
    select, bindparam, func, cast, text, literal_column, and_, or_, desc, union_all,
)
from sqlalchemy.orm import Session, registry, aliased, relationship
from sqlalchemy.dialects import postgresql, oracle

In [2]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
metadata_obj = MetaData()

In [3]:
user_table = Table(
    "user_account", metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

address_table = Table(
    "address", metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False)
)

metadata_obj.create_all(engine)

2022-10-03 17:23:50,014 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:23:50,017 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-10-03 17:23:50,019 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-03 17:23:50,020 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-10-03 17:23:50,022 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-03 17:23:50,023 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-10-03 17:23:50,024 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-03 17:23:50,025 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-10-03 17:23:50,025 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-03 17:23:50,027 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-10-03 17:23:50,029 INFO sqlalchemy.engine.Engine [no key 0.00172s] ()
2022-10-03 17:23:50,033 INFO sqlalchemy.engine.Engine 
C

In [4]:
stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"}
        ]
    )
    conn.commit()

2022-10-03 17:23:50,258 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:23:50,260 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-10-03 17:23:50,261 INFO sqlalchemy.engine.Engine [generated in 0.00301s] ('spongebob', 'Spongebob Squarepants')
2022-10-03 17:23:50,264 INFO sqlalchemy.engine.Engine COMMIT
2022-10-03 17:23:50,267 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:23:50,268 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-10-03 17:23:50,269 INFO sqlalchemy.engine.Engine [generated in 0.00231s] (('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star'))
2022-10-03 17:23:50,270 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
scalar_subq = (
    select(user_table.c.id).
    where(user_table.c.name == bindparam("username")).
    scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {"username": 'spongebob', "email_address": "spongebob@sqlalchemy.org"},
            {"username": 'sandy', "email_address": "sandy@sqlalchemy.org"},
            {"username": 'sandy', "email_address": "sandy@squirrelpower.org"},
        ]
    )
    conn.commit()

2022-10-03 17:23:50,490 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:23:50,491 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2022-10-03 17:23:50,492 INFO sqlalchemy.engine.Engine [generated in 0.00219s] (('spongebob', 'spongebob@sqlalchemy.org'), ('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org'))
2022-10-03 17:23:50,494 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
mapped_registry = registry()
Base = mapped_registry.generate_base()

In [7]:
class User(Base):
    __tablename__ = "user_account"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    
    addresses = relationship("Address", back_populates="user")
    
    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

In [8]:
class Address(Base):
    __tablename__ = "address"
    
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"))
    
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [9]:
mapped_registry.metadata.create_all(engine)

2022-10-03 17:23:51,444 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:23:51,446 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-10-03 17:23:51,447 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-03 17:23:51,450 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-10-03 17:23:51,451 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-03 17:23:51,452 INFO sqlalchemy.engine.Engine COMMIT


#### The `select()` SQL Expression Construct

The `select()` construct builds up a statement in the same way as that of `insert()`, using a _generative approach_ where each method builds more state onto the object. Like the other SQL constructs, it can be _stringified_ in place.

In [10]:
stmt = select(user_table).where(user_table.c.name == "spongebob")
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


Also in the same manner as all other statement-level `SQL constructs`, _to actually run the statement_ we pass it to an `execution` method. Since a `SELECT statement` returns _rows_ we can always iterate the result object to get _Row_ objects back.

In [11]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2022-10-03 17:23:51,912 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:23:51,915 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-10-03 17:23:51,916 INFO sqlalchemy.engine.Engine [generated in 0.00486s] ('spongebob',)
(1, 'spongebob', 'Spongebob Squarepants')
2022-10-03 17:23:51,919 INFO sqlalchemy.engine.Engine ROLLBACK


When using the ORM, particularly with a `select()` construct that's _composed against ORM entities_, we will want to execute it using the `Session.execute()` method on the `Session`; using this approach, we continue to get `Row` objects from the result, however these rows are now _capable of including **complete entities**_, such as instances of the `User` class, as individual elements within each row.

In [12]:
stmt = select(User).where(User.name == "spongebob")
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2022-10-03 17:23:52,120 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:23:52,129 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-10-03 17:23:52,131 INFO sqlalchemy.engine.Engine [generated in 0.00163s] ('spongebob',)
(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)
2022-10-03 17:23:52,133 INFO sqlalchemy.engine.Engine ROLLBACK


While the SQL generated in these examples looks the same whether we invoke `select(user_table)` or `select(User)`, in the more general case they _do not necessarily render the same thing_, as an ORM-mapped class may be mapped to other kinds of `"selectables"` besides tables. __The `select()` that's against an ORM entity also indicates that ORM-mapped instances should be returned in a result, which is not the case when SELECTing from a Table object.__

#### Setting the COLUMNS and FROM clause

The `select()` function accepts positional elements representing any number of `Column` and/or `Table` expressions, as well as a _wide range of compatible objects_, which are resolved into a __list of SQL expressions__ to be `SELECT`ed from that will be returned as columns in the result set. These elements also serve in simpler cases to create the `FROM` clause, which is inferred from the _columns and table-like expressions_ passed.

In [13]:
print(select(user_table))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


To `SELECT` from individual columns using a Core approach, `Column` objects are accessed from the _`Table.c` accessor_ and __can be sent directly__; the `FROM` clause will be inferred as the set of all `Table` and other `FromClause` objects that are represented by those columns.

In [14]:
print(select(user_table.c.name, user_table.c.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


##### Selecting ORM Entities and Columns

_ORM entities_, such our `User class` as well as the _column-mapped attributes upon it_ such as `User.name`, also participate in the `SQL Expression Language system` representing __tables and columns__. Below illustrates an example of `SELECT`ing from the `User` entity, which ultimately renders in the same way as if we had used `user_table` directly.

In [15]:
print(select(User))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


When executing a statement like the above using the _ORM `Session.execute()` method_, there is __an important difference__ when we _select from a full entity such as `User`, as opposed to `user_table`_, which is that the `entity` itself is returned as _a single element_ within each row. That is, when we fetch rows from the above statement, as there is only the User entity in the list of things to fetch, we get back `Row` objects that have only one element, which contain _instances of the `User` class_.

In [16]:
row = session.execute(select(User)).first()
print(row)

2022-10-03 17:23:52,935 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:23:52,937 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2022-10-03 17:23:52,938 INFO sqlalchemy.engine.Engine [generated in 0.00120s] ()
(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)


The above `Row` has just one element, representing the `User` entity.

In [17]:
print(row[0])

User(id=1, name='spongebob', fullname='Spongebob Squarepants')


A highly recommended convenience method of achieving the same result as above is to use the `Session.scalars()` method to _execute the statement **directly**_; this method will return a `ScalarResult` object that delivers the _first `column` of each row at once_, in this case, _instances of the `User` class_.

In [18]:
user = session.scalars(select(User)).first()
print(user)

2022-10-03 17:23:53,286 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2022-10-03 17:23:53,287 INFO sqlalchemy.engine.Engine [cached since 0.3501s ago] ()
User(id=1, name='spongebob', fullname='Spongebob Squarepants')


Alternatively, we can _select individual columns_ of an ORM entity as `distinct` elements within result rows, by _using the class-bound attributes_; when these are passed to a construct such as `select()`, they are resolved into the `Column` or other SQL expression represented by each attribute.

In [19]:
print(select(User.name, User.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


When we invoke this statement using `Session.execute()`, we now receive rows that have _individual elements per value_, each corresponding to a separate column or other SQL expression.

In [20]:
row = session.execute(select(User.name, User.fullname)).first()
print(row)

2022-10-03 17:23:53,596 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account
2022-10-03 17:23:53,598 INFO sqlalchemy.engine.Engine [generated in 0.00150s] ()
('spongebob', 'Spongebob Squarepants')


The approaches can also be mixed, as below where we `SELECT` the _name attribute_ of the User entity as the `first element of the row`, and **combine** it with _full Address entities_ in the `second element`.

In [21]:
session.execute(
    select(User.name, Address).
    where(User.id==Address.id).
    order_by(Address.id)
).all()

2022-10-03 17:23:53,773 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.id, address.email_address, address.user_id 
FROM user_account, address 
WHERE user_account.id = address.id ORDER BY address.id
2022-10-03 17:23:53,774 INFO sqlalchemy.engine.Engine [generated in 0.00121s] ()


[('spongebob', Address(id=1, email_address='spongebob@sqlalchemy.org')),
 ('sandy', Address(id=2, email_address='sandy@sqlalchemy.org')),
 ('patrick', Address(id=3, email_address='sandy@squirrelpower.org'))]

Approaches towards _selecting ORM entities and columns_ as well as common methods for _converting rows_ are discussed further at `Selecting ORM Entities and Attributes`.

##### Selecting from Labeled SQL Expressions

The `ColumnElement.label()` method as well as the _same-named method_ available on `ORM attributes` provides a _SQL label of a column or expression_, allowing it to have a **specific name** in a result set. This can be helpful when _referring to arbitrary SQL expressions in a result row by name_.

In [22]:
stmt = (
    select(
        ("Username: " + user_table.c.name).label("username"),
    ).order_by(user_table.c.name)
)

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.username}")

2022-10-03 17:23:53,979 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:23:53,980 INFO sqlalchemy.engine.Engine SELECT ? || user_account.name AS username 
FROM user_account ORDER BY user_account.name
2022-10-03 17:23:53,982 INFO sqlalchemy.engine.Engine [generated in 0.00259s] ('Username: ',)
Username: patrick
Username: sandy
Username: spongebob
2022-10-03 17:23:53,984 INFO sqlalchemy.engine.Engine ROLLBACK


##### Selecting with Textual Column Expressions

When we construct a `Select` object using the `select()` function, we are normally passing to it _a series of `Table` and `Column` objects_ that were defined using table metadata, or when using the ORM we may be sending `ORM-mapped attributes` that represent _table columns_. However, sometimes there is also the need to _manufacture arbitrary SQL blocks inside of statements_, such as `constant string expressions`, or just some arbitrary SQL that's quicker to write literally.

The `text()` construct introduced at `Working with Transactions and the DBAPI` can in fact be _embedded into a `Select` construct directly_, such as below where we manufacture a hardcoded string literal `'some label'` and __embed__ it within the `SELECT` statement.

In [23]:
stmt = (
    select(
        text("'some phrase'"), user_table.c.name
    ).order_by(user_table.c.name)
)

with engine.connect() as conn:
    print(conn.execute(stmt).all())

2022-10-03 17:23:54,152 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:23:54,153 INFO sqlalchemy.engine.Engine SELECT 'some phrase', user_account.name 
FROM user_account ORDER BY user_account.name
2022-10-03 17:23:54,154 INFO sqlalchemy.engine.Engine [generated in 0.00235s] ()
[('some phrase', 'patrick'), ('some phrase', 'sandy'), ('some phrase', 'spongebob')]
2022-10-03 17:23:54,157 INFO sqlalchemy.engine.Engine ROLLBACK


While the `text()` construct can be used in most places to _inject literal SQL phrases_, more often than not we are actually dealing with textual units that each represent an individual column expression. In this common case we can get more functionality out of our textual fragment using the `literal_column()` construct instead. This object is similar to `text()` except that _instead of representing arbitrary SQL of any form, it **explicitly represents `a single column`** and can then be `labeled and referred` towards in `subqueries and other expressions`_.

In [24]:
stmt = (
    select(
        literal_column("'some phrase'").label("p"), user_table.c.name,
    ).order_by(user_table.c.name)
)

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.p}, {row.name}")

2022-10-03 17:23:54,388 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:23:54,389 INFO sqlalchemy.engine.Engine SELECT 'some phrase' AS p, user_account.name 
FROM user_account ORDER BY user_account.name
2022-10-03 17:23:54,390 INFO sqlalchemy.engine.Engine [generated in 0.00264s] ()
some phrase, patrick
some phrase, sandy
some phrase, spongebob
2022-10-03 17:23:54,393 INFO sqlalchemy.engine.Engine ROLLBACK


Note that _in both cases_, when using `text()` or `literal_column()`, we are writing a **syntactical SQL expression**, and _not a literal value_. We therefore have to include whatever quoting or syntaxes are necessary for the SQL we want to see rendered.

#### The WHERE clause

SQLAlchemy allows us to _compose SQL expressions_, such as `name = 'squidward'` or `user_id > 10`, by making use of _standard `Python operators` in conjunction with `Column` and similar objects_. For `boolean expressions`, most Python operators such as `==, !=, <, >= etc.` _generate new SQL Expression objects, rather than plain boolean `True/False` values_.

In [25]:
print(user_table.c.name == "squidward")
print(address_table.c.user_id > 10)

user_account.name = :name_1
address.user_id > :user_id_1


We can use expressions like these to _generate the `WHERE` clause_ by passing the resulting objects to the `Select.where()` method.

In [26]:
print(select(user_table).where(user_table.c.name == "squidward"))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


To produce _multiple expressions `joined by AND`_, the `Select.where()` method may be _invoked any number of times_.

In [27]:
print(
    select(address_table.c.email_address).
    where(user_table.c.name == "squidward").
    where(address_table.c.user_id == user_table.c.id)
)

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


A single call to `Select.where()` also _accepts multiple expressions_ with the same effect.

In [28]:
print(
    select(address_table.c.email_address).
    where(
        user_table.c.name == "squidward",
        address_table.c.user_id == user_table.c.id
    )
)

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


`AND` and `OR` conjunctions are both __available directly__ using the `and_()` and `or_()` functions, illustrated below in terms of _ORM entities_.

In [29]:
print(
    select(Address.email_address).
    where(
        and_(
            or_(User.name == "squidward", User.name == "sandy"),
            Address.user_id == User.id
        )
    )
)

SELECT address.email_address 
FROM address, user_account 
WHERE (user_account.name = :name_1 OR user_account.name = :name_2) AND address.user_id = user_account.id


For *simple `"equality"` comparisons against a __single entity__*, there's also a popular method known as `Select.filter_by()` which accepts _keyword arguments that match to column keys or ORM attribute names_. It will filter against the _leftmost `FROM` clause_ or the _last entity joined_.

In [30]:
print(select(User).filter_by(name="spongebob", fullname="Spongebob Squarepants"))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1 AND user_account.fullname = :fullname_1


#### Explicit FROM clauses and JOINs

As mentioned previously, the `FROM` clause is usually _**inferred** based on the expressions_ that we are setting in the `columns` clause as well as other elements of the `Select`.

If we set a _single column_ from a particular `Table` in the `COLUMNS` clause, it puts that `Table` in the `FROM` clause as well.

In [31]:
print(select(user_table.c.name))

SELECT user_account.name 
FROM user_account


If we were to put _columns from two tables_, then we get a `comma-separated FROM clause`.

In [32]:
print(select(user_table.c.name, address_table.c.email_address))

SELECT user_account.name, address.email_address 
FROM user_account, address


In order to `JOIN` these two tables together, we typically use one of two methods on `Select`. The first is the `Select.join_from()` method, which _allows us to indicate the **left and right** side of the JOIN **explicitly**_.

In [33]:
print(
    select(user_table.c.name, address_table.c.email_address).
    join_from(user_table, address_table)
)

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


The other is the the `Select.join()` method, which indicates only the **right side** of the `JOIN`, the _left hand-side is inferred_.

In [34]:
print(
    select(user_table.c.name, address_table.c.email_address).
    join(address_table)
)

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


> ###### The ON Clause is inferred
>
> When using `Select.join_from()` or `Select.join()`, we may observe that the `ON clause` of the join is also __inferred__ for us in _simple foreign key cases_.

We also have the _option to add elements to the `FROM` clause explicitly_, if it is not inferred the way we want from the columns clause. We use the `Select.select_from()` method to achieve this, as below where we establish `user_table` as the first element in the FROM clause and `Select.join()` to establish `address_table` as the second.

In [35]:
print(
    select(address_table.c.email_address).
    select_from(user_table).join(address_table)
)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


Another example where we might want to use `Select.select_from()` is if our columns clause _doesn't have enough information_ to provide for a `FROM` clause. For example, to `SELECT` from the common SQL expression `count(*)`, we use a SQLAlchemy element known as `sqlalchemy.sql.expression.func` to produce the SQL `count()` function.

In [36]:
print(select(func.count("*")).select_from(user_table))

SELECT count(:count_2) AS count_1 
FROM user_account


##### Setting the ON Clause

The previous examples of `JOIN` illustrated that the `Select` construct _can join between two tables and produce the ON clause automatically_. This occurs in those examples because _the `user_table` and `address_table` Table objects include **a single `ForeignKeyConstraint`** definition_ which is used to form this `ON` clause.

If the _`left and right targets of the join` do not have such a constraint_, or there are _multiple constraints_ in place, we __need to specify the `ON` clause directly__. Both `Select.join()` and `Select.join_from()` accept an additional argument for the `ON` clause, which is stated using the same SQL Expression mechanics as we saw about in The `WHERE` clause.

In [37]:
print(
    select(address_table.c.email_address).
    select_from(user_table).
    join(address_table, user_table.c.id == address_table.c.user_id)
)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


> **ORM Tip** - there's another way to generate the `ON` clause when _using ORM entities_ that make use of the `relationship()` construct, like the mapping set up in the previous section at `Declaring Mapped Classes`. This is a whole subject onto itself, which is introduced at length at Using Relationships to Join.

##### OUTER and FULL join

Both the `Select.join()` and `Select.join_from()` methods accept keyword arguments `Select.join.isouter` and `Select.join.full` which will render **`LEFT OUTER JOIN`** and **`FULL OUTER JOIN`**, respectively.

In [38]:
print(select(user_table).join(address_table, isouter=True))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account LEFT OUTER JOIN address ON user_account.id = address.user_id


In [39]:
print(select(user_table).join(address_table, full=True))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account FULL OUTER JOIN address ON user_account.id = address.user_id


There is also a method `Select.outerjoin()` that is equivalent to using `.join(..., isouter=True)`.

> `SQL` also has a **`RIGHT OUTER JOIN`**. __`SQLAlchemy` doesn't render this directly__; instead, __reverse the order of the tables and use `LEFT OUTER JOIN`__.

#### ORDER BY, GROUP BY, HAVING

The `SELECT SQL statement` includes a clause called `ORDER BY` which is used to _return the selected rows within a given ordering_.

The `GROUP BY` clause is constructed similarly to the `ORDER BY` clause, and has the purpose of _sub-dividing the selected rows into specific groups upon which aggregate functions_ may be invoked. _The `HAVING` clause is usually used with `GROUP BY`_ and is of a similar form to the `WHERE` clause, except that it's _applied to the aggregated functions used within groups_.

#### ORDER BY

The `ORDER BY` clause is constructed in terms of SQL Expression constructs typically based on `Column` or _similar objects_. The `Select.order_by()` method accepts _one or more of these expressions_ **positionally**.

In [40]:
print(select(user_table).order_by(user_table.c.name))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.name


`Ascending/descending` is available from the `ColumnElement.asc()` and `ColumnElement.desc()` modifiers, which are present from _ORM-bound attributes_ as well.

In [41]:
print(select(User).order_by(User.fullname.desc()))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.fullname DESC


The above statement will yield rows that are `sorted` by the `user_account.fullname` column in _descending order_.

#### Aggregate functions with `GROUP BY/HAVING`

In `SQL`, `aggregate functions` allow column expressions across multiple rows to be aggregated together to produce a single result. Examples include `counting`, `computing averages`, as well as locating the `maximum` or `minimum` value in a set of values.

SQLAlchemy provides for SQL functions in an _open-ended way_ using a namespace known as `func`. This is a _special constructor_ object which will _create new instances_ of `Function` when given the name of a particular SQL function, which can have _any name_, as well as _zero or more arguments_ to pass to the `function`, which are, like in all other cases, SQL Expression constructs. For example, to render the `SQL COUNT()` function against the `user_account.id` column, we call upon the `count()` name.

In [42]:
count_fn = func.count(user_table.c.id)
print(count_fn)

count(user_account.id)


When using `aggregate functions` in `SQL`, the `GROUP BY` clause is __essential__ in that it `allows rows to be partitioned into groups` where _aggregate functions_ will be applied to each group __individually__. When requesting `non-aggregated columns` in the `COLUMNS` clause of a `SELECT` statement, SQL requires that these columns all be subject to a `GROUP BY` clause, either __directly or indirectly__ based on a `primary key association`. The `HAVING` clause is then used in a similar manner as the `WHERE` clause, except that it _filters out rows based on aggregated values_ rather than direct row contents.

SQLAlchemy provides for these _two clauses_ using the `Select.group_by()` and `Select.having()` methods. Below we illustrate selecting user name fields as well as count of addresses, for those users that have more than one address.

In [43]:
with engine.connect() as conn:
    result = conn.execute(
        select(User.name, func.count(Address.id).label("count")).
        join(Address).group_by(User.name).having(func.count(Address.id) > 1)
    )
    print(result.all())

2022-10-03 17:23:57,904 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:23:57,905 INFO sqlalchemy.engine.Engine SELECT user_account.name, count(address.id) AS count 
FROM user_account JOIN address ON user_account.id = address.user_id GROUP BY user_account.name 
HAVING count(address.id) > ?
2022-10-03 17:23:57,906 INFO sqlalchemy.engine.Engine [generated in 0.00250s] (1,)
[('sandy', 2)]
2022-10-03 17:23:57,909 INFO sqlalchemy.engine.Engine ROLLBACK


#### Ordering or Grouping by a Label

An important technique, in particular on some database backends, is the ability to `ORDER BY` or `GROUP BY` an expression that is already stated in the columns clause, __without re-stating__ the expression in the `ORDER BY` or `GROUP BY` clause and instead using the _column name_ or _labeled name_ from the `COLUMNS` clause. This form is available by passing the __string text of the name__ to the `Select.order_by()` or `Select.group_by()` method. The text passed is __not rendered directly__; instead, the name given to an expression in the columns clause and rendered as that expression name in context, raising an error if no match is found. The unary modifiers `asc()` and `desc()` may also be used in this form.

In [44]:
stmt = select(
    Address.user_id, func.count(Address.id).label("num_addresses")
).group_by("user_id").order_by("user_id", desc("num_addresses"))
print(stmt)

SELECT address.user_id, count(address.id) AS num_addresses 
FROM address GROUP BY address.user_id ORDER BY address.user_id, num_addresses DESC


#### Using Aliases

Now that we are selecting from `multiple tables` and using __joins__, we quickly run into the case where we need to _refer to the same table `multiple` times_ in the `FROM` clause of a statement. We accomplish this using `SQL aliases`, which are a syntax that supplies an __alternative name__ to a `table or subquery` from which it can be referred towards in the statement.

In the SQLAlchemy Expression Language, these __names__ are instead represented by `FromClause` objects known as the __`Alias construct`__, which is constructed in `Core` using the `FromClause.alias()` method. An `Alias construct` is just like a `Table` construct in that it also has a _namespace of `Column` objects_ within the `Alias.c` collection. The `SELECT` statement below for example returns all unique pairs of user names.

In [45]:
user_alias_1 = user_table.alias()
user_alias_2 = user_table.alias()

print(
    select(user_alias_1.c.name, user_alias_2.c.name).
    join_from(user_alias_1, user_alias_2, user_alias_1.c.id > user_alias_2.c.id)
)

SELECT user_account_1.name, user_account_2.name AS name_1 
FROM user_account AS user_account_1 JOIN user_account AS user_account_2 ON user_account_1.id > user_account_2.id


#### ORM Entity Aliases

The ORM equivalent of the `FromClause.alias()` method is the ORM `aliased()` function, which may be applied to an entity such as `User` and `Address`. This produces a `Alias` object internally that's against the original mapped `Table` object, while maintaining ORM functionality. The `SELECT` below selects from the `User` entity all objects that include two particular email addresses.

In [46]:
address_alias_1 = aliased(Address)
address_alias_2 = aliased(Address)

print(
    select(User).
    join_from(User, address_alias_1).
    where(address_alias_1.email_address == "patrick@aol.com").
    join_from(User, address_alias_2).
    where(address_alias_2.email_address == "patrick@gmail.com")
)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account JOIN address AS address_1 ON user_account.id = address_1.user_id JOIN address AS address_2 ON user_account.id = address_2.user_id 
WHERE address_1.email_address = :email_address_1 AND address_2.email_address = :email_address_2


As mentioned in `Setting` the `ON Clause`, the ORM provides for another way to join using the `relationship()` construct. The above example using `aliases` is demonstrated using `relationship()` at _Joining between Aliased targets_.

#### Subqueries and CTEs

A __`subquery`__ in SQL is a `SELECT` statement that is _rendered within parenthesis_ and _placed within the context of an enclosing statement_, _typically_ a `SELECT` statement but _not necessarily_.

This section will cover a so-called __non-scalar__ `subquery`, which is _typically placed in the FROM clause of an enclosing SELECT_. We will also cover the `Common Table Expression` or `CTE`, which is used in a similar way as a subquery, but includes _additional features_.

SQLAlchemy uses the `Subquery` object to represent a _subquery_ and the `CTE` to represent a _CTE_, usually obtained from the `Select.subquery()` and `Select.cte()` methods, respectively. Either object can be used as a `FROM` element `inside of a larger select()` construct.

We can construct a `Subquery` that will _select an aggregate count of rows_ from the address table (`aggregate functions` and `GROUP BY` were introduced previously at `Aggregate functions with GROUP BY/HAVING`).

In [47]:
subq = select(
    func.count(address_table.c.id).label("count"),
    address_table.c.user_id,
).group_by(address_table.c.user_id).subquery()

print(subq)

SELECT count(address.id) AS count, address.user_id 
FROM address GROUP BY address.user_id


The `Subquery` object behaves like any other `FROM` object such as a `Table`, notably that it includes a `Subquery.c` namespace of the columns which it selects. We can use this namespace to refer to both the `user_id` column as well as our custom labeled `count` expression.

In [48]:
print(select(subq.c.user_id, subq.c.count))

SELECT anon_1.user_id, anon_1.count 
FROM (SELECT count(address.id) AS count, address.user_id AS user_id 
FROM address GROUP BY address.user_id) AS anon_1


With a selection of rows contained within the _subq_ object, we can _apply the object to a larger `Select`_ that will join the data to the `user_account` table.

In [49]:
stmt = select(
    user_table.c.name,
    user_table.c.fullname,
    subq.c.count,
).join_from(user_table, subq)

print(stmt)

SELECT user_account.name, user_account.fullname, anon_1.count 
FROM user_account JOIN (SELECT count(address.id) AS count, address.user_id AS user_id 
FROM address GROUP BY address.user_id) AS anon_1 ON user_account.id = anon_1.user_id


In order to __join__ from `user_account` to `address`, we made use of the `Select.join_from()` method. As has been illustrated previously, the `ON clause` of this _join_ was again __inferred__ based on _foreign key constraints_. Even though a `SQL subquery` __does not itself have any constraints__, SQLAlchemy can act upon constraints represented on the columns by determining that the `subq.c.user_id` column is _derived from_ the `address_table.c.user_id` column, which __does express__ a `foreign key relationship` back to the `user_table.c.id` column which is then used to _generate_ the `ON clause`.

#### Common Table Expressions (CTEs)

Usage of the `CTE` construct in SQLAlchemy is _virtually the same_ as how the `Subquery` construct is used. By changing the invocation of the `Select.subquery()` method to use `Select.cte()` instead, we can use the resulting object as a `FROM` element in the same way, but the __SQL rendered is the very different__ common table expression syntax.

In [50]:
subq = select(
    func.count(address_table.c.id).label("count"),
    address_table.c.user_id
).group_by(address_table.c.user_id).cte()

stmt = select(
    user_table.c.name,
    user_table.c.fullname,
    subq.c.count,
).join_from(user_table, subq)
print(stmt)

WITH anon_1 AS 
(SELECT count(address.id) AS count, address.user_id AS user_id 
FROM address GROUP BY address.user_id)
 SELECT user_account.name, user_account.fullname, anon_1.count 
FROM user_account JOIN anon_1 ON user_account.id = anon_1.user_id


The `CTE` construct also features the ability to be used in a __recursive style__, and may in more elaborate cases be __composed from the RETURNING clause__ of an `INSERT, UPDATE or DELETE` statement. The docstring for `CTE` includes details on these additional patterns.

In both cases, the `subquery` and `CTE` were named at the SQL level using an __anonymous__ name. In the Python code, we don't need to provide these names at all. The __object identity__ of the `Subquery` or `CTE` instances serves as the __syntactical identity__ of the object when rendered. A _name that will be rendered_ in the SQL can be provided by passing it as the _first argument_ of the `Select.subquery()` or `Select.cte()` methods.

#### ORM Entity `Subqueries/CTEs`

In the ORM, the `aliased()` construct may be used to associate an ORM entity, such as our `User` or `Address` class, with any `FromClause` concept that represents a _source of rows_. The preceding section `ORM Entity Aliases` illustrates using `aliased()` to __associate the mapped class with an `Alias` of its mapped `Table`__. Here we illustrate `aliased()` doing the same thing against both a `Subquery` as well as a `CTE` generated against a `Select` construct, that ultimately derives from that same mapped `Table`.

Below is an example of applying `aliased()` to the `Subquery` construct, so that _ORM entities can be extracted from its rows_. The result shows a series of `User` and `Address` objects, where the data for each `Address` object ultimately came from a __subquery__ against the `address` table rather than that `table` directly.

In [51]:
subq = select(Address).where(~Address.email_address.like("%@aol.com")).subquery()
address_subq = aliased(Address, subq)
stmt = select(User, address_subq).join_from(User, address_subq).order_by(User.id, address_subq.id)

with Session(engine) as session:
    for user, address in session.execute(stmt):
        print(f"{user=} {address=}")

2022-10-03 17:23:59,480 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:23:59,484 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname, anon_1.id AS id_1, anon_1.email_address, anon_1.user_id 
FROM user_account JOIN (SELECT address.id AS id, address.email_address AS email_address, address.user_id AS user_id 
FROM address 
WHERE address.email_address NOT LIKE ?) AS anon_1 ON user_account.id = anon_1.user_id ORDER BY user_account.id, anon_1.id
2022-10-03 17:23:59,486 INFO sqlalchemy.engine.Engine [generated in 0.00142s] ('%@aol.com',)
user=User(id=1, name='spongebob', fullname='Spongebob Squarepants') address=Address(id=1, email_address='spongebob@sqlalchemy.org')
user=User(id=2, name='sandy', fullname='Sandy Cheeks') address=Address(id=2, email_address='sandy@sqlalchemy.org')
user=User(id=2, name='sandy', fullname='Sandy Cheeks') address=Address(id=3, email_address='sandy@squirrelpower.org')
2022-10-03 17:23:59,489 INFO sqlalchemy.e

Another example follows, which is __exactly the same__ except it makes use of the `CTE` construct instead.

In [52]:
cte_obj = select(Address).where(~Address.email_address.like("%@aol.com")).cte()
address_cte = aliased(Address, cte_obj)
stmt = select(User, address_cte).join_from(User, address_cte).order_by(User.id, address_cte.id)

with Session(engine) as session:
    for user, address in session.execute(stmt):
        print(f"{user=} {address=}")

2022-10-03 17:23:59,684 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:23:59,688 INFO sqlalchemy.engine.Engine WITH anon_1 AS 
(SELECT address.id AS id, address.email_address AS email_address, address.user_id AS user_id 
FROM address 
WHERE address.email_address NOT LIKE ?)
 SELECT user_account.id, user_account.name, user_account.fullname, anon_1.id AS id_1, anon_1.email_address, anon_1.user_id 
FROM user_account JOIN anon_1 ON user_account.id = anon_1.user_id ORDER BY user_account.id, anon_1.id
2022-10-03 17:23:59,689 INFO sqlalchemy.engine.Engine [generated in 0.00126s] ('%@aol.com',)
user=User(id=1, name='spongebob', fullname='Spongebob Squarepants') address=Address(id=1, email_address='spongebob@sqlalchemy.org')
user=User(id=2, name='sandy', fullname='Sandy Cheeks') address=Address(id=2, email_address='sandy@sqlalchemy.org')
user=User(id=2, name='sandy', fullname='Sandy Cheeks') address=Address(id=3, email_address='sandy@squirrelpower.org')
2022-10-03 17:23:59,693 INF

#### Scalar and Correlated Subqueries

A `scalar subquery` is a _subquery_ that returns __exactly zero or one row and exactly one column__. The _subquery_ is then used in the `COLUMNS` or `WHERE` clause of an `enclosing SELECT statement` and is __different than a regular subquery__ in that it is __not used in the FROM clause__. A `correlated subquery` is a _scalar subquery_ that __refers to a table__ in the `enclosing SELECT statement`.

SQLAlchemy represents the _scalar subquery_ using the `ScalarSelect` construct, which is part of the __`ColumnElement` expression hierarchy__, in __contrast__ to the _regular subquery_ which is represented by the `Subquery` construct, which is in the `FromClause` hierarchy.

`Scalar subqueries` are __often, but not necessarily__, `used with aggregate functions`, introduced previously at `Aggregate functions with GROUP BY/HAVING`. A _scalar subquery_ is __indicated explicitly__ by making use of the `Select.scalar_subquery()` method as below. It's _default string form_ when `stringified` by itself _renders as an ordinary SELECT statement_ that is selecting from two tables.

In [53]:
subq = select(
    func.count(address_table.c.id)
).where(user_table.c.id == address_table.c.user_id).scalar_subquery()
print(subq)

(SELECT count(address.id) AS count_1 
FROM address, user_account 
WHERE user_account.id = address.user_id)


The above `subq` object now __falls within__ the `ColumnElement` SQL expression hierarchy, in that it _may be used like any other column expression_.

In [54]:
print(subq == 5)

(SELECT count(address.id) AS count_1 
FROM address, user_account 
WHERE user_account.id = address.user_id) = :param_1


Although the _scalar subquery_ by itself renders both `user_account` and `address` in its `FROM clause` when _stringified by itself_, when __embedding__ it into an `enclosing select() construct` that deals with the `user_account` table, the `user_account` table is __automatically correlated__, meaning it __does not render__ in the `FROM clause` of the subquery.

In [55]:
stmt = select(user_table.c.name, subq.label("address_count"))
print(stmt)

SELECT user_account.name, (SELECT count(address.id) AS count_1 
FROM address 
WHERE user_account.id = address.user_id) AS address_count 
FROM user_account


`Simple correlated subqueries` will usually __do the right thing__ that's desired. However, in the case where the _correlation is ambiguous_, SQLAlchemy will let us know that __`more clarity is needed`__.

In [56]:
try:
    stmt = select(
        user_table.c.name,
        address_table.c.email_address,
        subq.label("address_count"),
    ).join_from(user_table, address_table).order_by(user_table.c.id, address_table.c.id)
    print(stmt)
except Exception as e:
    print(str(e))

Select statement '<sqlalchemy.sql.selectable.Select object at 0x0000010CBFC34E20>' returned no FROM clauses due to auto-correlation; specify correlate(<tables>) to control correlation manually.


To specify that the `user_table` is the one we seek to __correlate__ we specify this using the `ScalarSelect.correlate()` or `ScalarSelect.correlate_except()` methods. The statement then can return the data for this column like any other.

In [57]:
subq = select(
    func.count(address_table.c.id)
).where(user_table.c.id == address_table.c.user_id).\
scalar_subquery().correlate(user_table)

with engine.connect() as conn:
    result = conn.execute(
        select(
            user_table.c.name,
            address_table.c.email_address,
            subq.label("address_count")
        ).
        join_from(user_table, address_table).
        order_by(user_table.c.id, address_table.c.id)
    )

print(result.all())

2022-10-03 17:24:00,651 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:24:00,652 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.email_address, (SELECT count(address.id) AS count_1 
FROM address 
WHERE user_account.id = address.user_id) AS address_count 
FROM user_account JOIN address ON user_account.id = address.user_id ORDER BY user_account.id, address.id
2022-10-03 17:24:00,653 INFO sqlalchemy.engine.Engine [generated in 0.00273s] ()
2022-10-03 17:24:00,655 INFO sqlalchemy.engine.Engine ROLLBACK
[('spongebob', 'spongebob@sqlalchemy.org', 1), ('sandy', 'sandy@sqlalchemy.org', 2), ('sandy', 'sandy@squirrelpower.org', 2)]


##### LATERAL correlation

`LATERAL correlation` is a _special sub-category of SQL correlation_ which __allows a selectable unit to refer to another selectable unit within a single `FROM clause`__. This is an `extremely special use case` which, while part of the SQL standard, is only known to be supported by __recent versions of PostgreSQL__.

Normally, if a `SELECT` statement refers to `table1 JOIN (SELECT ...) AS subquery` in its `FROM clause`, the `subquery on the right side` __may not refer to the `table1` expression from the left side__; `correlation` __may only refer to a table that is part of another SELECT that entirely encloses this SELECT__. The `LATERAL` keyword allows us to turn this behavior around and __allow correlation from the right side JOIN__.

SQLAlchemy supports this feature using the `Select.lateral()` method, which creates an object known as `Lateral`. `Lateral` is in the same family as `Subquery` and `Alias`, but also __includes correlation behavior__ when the construct is added to the `FROM clause` of an _enclosing SELECT_. The following example illustrates a SQL query that makes use of `LATERAL`, selecting the `"user account/count of email address"` data as was discussed in the previous section.

In [58]:
subq = (
    select(
        func.count(address_table.c.id).label("address_count"),
        address_table.c.email_address,
        address_table.c.user_id,
    ).
    where(user_table.c.id == address_table.c.user_id).
    lateral()
)

stmt = (
    select(
        user_table.c.name,
        subq.c.address_count,
        subq.c.email_address,
    ).
    join_from(user_table, subq).
    order_by(user_table.c.id, subq.c.email_address)
)

print(stmt)

SELECT user_account.name, anon_1.address_count, anon_1.email_address 
FROM user_account JOIN LATERAL (SELECT count(address.id) AS address_count, address.email_address AS email_address, address.user_id AS user_id 
FROM address 
WHERE user_account.id = address.user_id) AS anon_1 ON user_account.id = anon_1.user_id ORDER BY user_account.id, anon_1.email_address


Above, the _right side_ of the `JOIN` is a `subquery` that __correlates__ to the `user_account` table that's on the _left side_ of the join.

When using `Select.lateral()`, the behavior of `Select.correlate()` and `Select.correlate_except()` methods is applied to the `Lateral` construct as well.

#### `UNION`, `UNION ALL` and other set operations

In SQL, `SELECT` statements __can be merged together__ using the `UNION` or `UNION ALL` SQL operation, which produces the _set of all rows produced by one or more statements together_. Other _set operations_ such as `INTERSECT [ALL]` and `EXCEPT [ALL]` are also possible.

SQLAlchemy's `Select` construct __supports compositions__ of this nature using functions like `union()`, `intersect()` and `except_()`, and the all counterparts `union_all()`, `intersect_all()` and `except_all()`. These functions all _accept an arbitrary number of sub-selectables_, which are typically `Select` constructs but _may also be an existing composition_.

The construct produced by these functions is the `CompoundSelect`, which is used in the same manner as the `Select` construct, except that it has __fewer methods__. The `CompoundSelect` produced by `union_all()` for example may be __invoked directly using Connection.execute()__.

In [59]:
stmt1 = select(user_table).where(user_table.c.name == "sandy")
stmt2 = select(user_table).where(user_table.c.name == "spongebob")
u = union_all(stmt1, stmt2)

with engine.connect() as conn:
    result = conn.execute(u)
    print(result.all())

2022-10-03 17:24:01,130 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:24:01,131 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ? UNION ALL SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-10-03 17:24:01,132 INFO sqlalchemy.engine.Engine [generated in 0.00294s] ('sandy', 'spongebob')
[(2, 'sandy', 'Sandy Cheeks'), (1, 'spongebob', 'Spongebob Squarepants')]
2022-10-03 17:24:01,134 INFO sqlalchemy.engine.Engine ROLLBACK


To use a `CompoundSelect` as a _subquery_, just like `Select` it provides a `SelectBase.subquery()` method which will produce a `Subquery` object with a `FromClause.c collection` that __may be referred__ towards in an `enclosing select()`.

In [60]:
u_subq = u.subquery()
stmt = (
    select(u_subq.c.name, address_table.c.email_address).
    join_from(address_table, u_subq).
    order_by(u_subq.c.name, address_table.c.email_address)
)

with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2022-10-03 17:24:01,278 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:24:01,280 INFO sqlalchemy.engine.Engine SELECT anon_1.name, address.email_address 
FROM address JOIN (SELECT user_account.id AS id, user_account.name AS name, user_account.fullname AS fullname 
FROM user_account 
WHERE user_account.name = ? UNION ALL SELECT user_account.id AS id, user_account.name AS name, user_account.fullname AS fullname 
FROM user_account 
WHERE user_account.name = ?) AS anon_1 ON anon_1.id = address.user_id ORDER BY anon_1.name, address.email_address
2022-10-03 17:24:01,281 INFO sqlalchemy.engine.Engine [generated in 0.00311s] ('sandy', 'spongebob')
[('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org'), ('spongebob', 'spongebob@sqlalchemy.org')]
2022-10-03 17:24:01,282 INFO sqlalchemy.engine.Engine ROLLBACK


##### Selecting ORM Entities from Unions

The preceding examples illustrated how to construct a `UNION` given two `Table` objects, to then return database rows. If we wanted to use a `UNION` or _other set operation_ to select rows that we then receive as ORM objects, there are __two approaches__ that may be used. In both cases, we first construct a `select()` or `CompoundSelect` object that represents the `SELECT/UNION/etc statement` we want to execute; this statement should be _composed against the target ORM entities_ or their _underlying mapped Table objects_.

In [61]:
stmt1 = select(User).where(User.name == "sandy")
stmt2 = select(User).where(User.name == "spongebob")
u = union_all(stmt1, stmt2)
print(u)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1 UNION ALL SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_2


For a `simple SELECT with UNION` that is _not already nested inside of a subquery_, these can often be used in an _ORM object fetching context_ by using the `Select.from_statement()` method. With this approach, the `UNION` statement represents the entire query; __no additional criteria can be added after `Select.from_statement()` is used__.

In [62]:
orm_stmt = select(User).from_statement(u)

with Session(engine) as session:
    for obj in session.execute(orm_stmt).scalars():
        print(obj)

2022-10-03 17:24:01,618 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:24:01,621 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ? UNION ALL SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-10-03 17:24:01,623 INFO sqlalchemy.engine.Engine [generated in 0.00208s] ('sandy', 'spongebob')
User(id=2, name='sandy', fullname='Sandy Cheeks')
User(id=1, name='spongebob', fullname='Spongebob Squarepants')
2022-10-03 17:24:01,626 INFO sqlalchemy.engine.Engine ROLLBACK


To use a `UNION` or `other set-related construct` as an _entity-related component_ in in a more flexible manner, the `CompoundSelect` construct may be organized into a `subquery` using `CompoundSelect.subquery()`, which then __links to ORM objects using the aliased()__ function. This works in the same way introduced at `ORM Entity Subqueries/CTEs`, to first __create an ad-hoc `"mapping"`__ of our desired entity to the subquery, then selecting from that that new entity as though it were any other mapped class. In the example below, we are able to _add additional criteria_ such as `ORDER BY outside of the UNION itself`, as we can `filter` or `order by` the columns exported by the subquery.

In [63]:
user_alias = aliased(User, u.subquery())
orm_stmt = select(user_alias).order_by(user_alias.id)

with Session(engine) as session:
    for obj in session.execute(orm_stmt).scalars():
        print(obj)

2022-10-03 17:24:01,778 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:24:01,785 INFO sqlalchemy.engine.Engine SELECT anon_1.id, anon_1.name, anon_1.fullname 
FROM (SELECT user_account.id AS id, user_account.name AS name, user_account.fullname AS fullname 
FROM user_account 
WHERE user_account.name = ? UNION ALL SELECT user_account.id AS id, user_account.name AS name, user_account.fullname AS fullname 
FROM user_account 
WHERE user_account.name = ?) AS anon_1 ORDER BY anon_1.id
2022-10-03 17:24:01,786 INFO sqlalchemy.engine.Engine [generated in 0.00124s] ('sandy', 'spongebob')
User(id=1, name='spongebob', fullname='Spongebob Squarepants')
User(id=2, name='sandy', fullname='Sandy Cheeks')
2022-10-03 17:24:01,788 INFO sqlalchemy.engine.Engine ROLLBACK


#### `EXISTS` subqueries

The SQL `EXISTS` keyword is an operator that is __used with scalar subqueries__ to _return a boolean true or false_ depending on if the `SELECT` statement would return a row. SQLAlchemy includes a variant of the `ScalarSelect` object called `Exists`, which will __generate an EXISTS subquery__ and is most conveniently generated using the `SelectBase.exists()` method. Below we produce an `EXISTS` so that we can return `user_account` rows that have more than one related row in `address`.

In [64]:
subq = (
    select(func.count(address_table.c.id)).
    where(user_table.c.id == address_table.c.user_id).
    group_by(address_table.c.user_id).
    having(func.count(address_table.c.id) > 1)
).exists()

with engine.connect() as conn:
    result = conn.execute(select(user_table.c.name).where(subq))
    print(result.all())

2022-10-03 17:24:01,974 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:24:01,975 INFO sqlalchemy.engine.Engine SELECT user_account.name 
FROM user_account 
WHERE EXISTS (SELECT count(address.id) AS count_1 
FROM address 
WHERE user_account.id = address.user_id GROUP BY address.user_id 
HAVING count(address.id) > ?)
2022-10-03 17:24:01,976 INFO sqlalchemy.engine.Engine [generated in 0.00315s] (1,)
[('sandy',)]
2022-10-03 17:24:01,979 INFO sqlalchemy.engine.Engine ROLLBACK


The `EXISTS` construct is _more often_ than not __used as a negation__, e.g. `NOT EXISTS`, as it provides a _SQL-efficient form of locating rows_ for which a related table has no rows. Below we select user names that have no email addresses; note the __binary negation operator (~)__ used inside the second `WHERE clause`.

In [65]:
subq = (
    select(address_table.c.id).
    where(user_table.c.id == address_table.c.user_id)
).exists()

with engine.connect() as conn:
    result = conn.execute(select(user_table.c.name).where(~subq))
    print(result.all())

2022-10-03 17:24:02,186 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:24:02,187 INFO sqlalchemy.engine.Engine SELECT user_account.name 
FROM user_account 
WHERE NOT (EXISTS (SELECT address.id 
FROM address 
WHERE user_account.id = address.user_id))
2022-10-03 17:24:02,188 INFO sqlalchemy.engine.Engine [generated in 0.00220s] ()
[('patrick',)]
2022-10-03 17:24:02,190 INFO sqlalchemy.engine.Engine ROLLBACK


#### Working with SQL Functions

First introduced earlier in this section at `Aggregate functions with GROUP BY/HAVING`, the `func` object serves as a __factory for creating new Function objects__, which when used in a construct like `select()`, _produce a SQL function display_, typically consisting of a `name`, `some parenthesis` (although __not always__), and possibly `some arguments`. Examples of _typical SQL functions_ include:

* the `count()` function, an _aggregate function_ which __counts how many rows__ are returned.

In [66]:
print(select(func.count()).select_from(user_table))

SELECT count(*) AS count_1 
FROM user_account


* the `lower()` function, a _string function_ that __converts a string to lower case__.

In [67]:
print(select(func.lower("A String With Much UPPERCASE")))

SELECT lower(:lower_2) AS lower_1


* the `now()` function, which provides for the __current date and time__; as this is a common function, `SQLAlchemy` _knows_ how to __render this differently for each backend__, in the case of `SQLite` using the `CURRENT_TIMESTAMP` function.

In [68]:
stmt = select(func.now())

with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2022-10-03 17:24:02,755 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:24:02,757 INFO sqlalchemy.engine.Engine SELECT CURRENT_TIMESTAMP AS now_1
2022-10-03 17:24:02,758 INFO sqlalchemy.engine.Engine [generated in 0.00295s] ()
[(datetime.datetime(2022, 10, 3, 11, 24, 2),)]
2022-10-03 17:24:02,760 INFO sqlalchemy.engine.Engine ROLLBACK


As _most database backends_ feature __dozens if not hundreds__ of `different SQL functions`, `func` tries to be __as liberal as possible__ in what it accepts. __Any name__ that is _accessed_ from this `namespace` is __automatically considered to be a SQL function__ that will _render in a generic way_.

In [69]:
print(select(func.some_crazy_function(user_table.c.name, 17)))

SELECT some_crazy_function(user_account.name, :some_crazy_function_2) AS some_crazy_function_1 
FROM user_account


At the same time, a _relatively small set of extremely common SQL functions_ such as `count`, `now`, `max`, `concat` include __pre-packaged versions__ of themselves which provide for __proper typing information__ as well as __backend-specific SQL generation__ in some cases. The example below contrasts the SQL generation that occurs for the `PostgreSQL` dialect compared to the `Oracle` dialect for the `now` function.

In [70]:
print(select(func.now()).compile(dialect=postgresql.dialect()))

SELECT now() AS now_1


In [71]:
print(select(func.now()).compile(dialect=oracle.dialect()))

SELECT CURRENT_TIMESTAMP AS now_1 FROM DUAL


##### Functions Have Return Types

As __functions are column expressions__, they also have `SQL datatypes` that _describe the data type of a generated SQL expression_. We refer to these types here as `"SQL return types"`, in reference to the type of `SQL value` that is returned by the function in the context of a _database-side SQL expression_, as opposed to the `"return type"` of a Python function.

The `SQL return type` of any SQL function _may be accessed, typically for debugging purposes_, by referring to the `Function.type` attribute.

In [72]:
func.now().type

DateTime()

These `SQL return types` are __significant__ when making use of the function expression in the context of a larger expression; that is, `math` operators will _work better_ when the `datatype of the expression` is something like __`Integer`__ or __`Numeric`__, _JSON accessors_ in order to work need to be using a type such as `JSON`. _Certain classes_ of functions __return entire rows__ instead of column values, where there is a _need to refer to specific columns_; such functions are referred towards as `table valued functions`.

The `SQL return type` of the function __may also be significant__ when _executing a statement and getting rows back_, for those cases where SQLAlchemy has to __apply result-set processing__. A prime example of this are _date-related functions on SQLite_, where SQLAlchemy's `DateTime and related datatypes` take on the role of __converting from string values to Python datetime() objects__ as result rows are received.

To __apply a `specific type` to a function__ we're creating, we pass it using the `Function.type_` parameter; the _type argument_ may be either a `TypeEngine class or an instance`. In the example below we pass the `JSON` class to generate the `PostgreSQL json_object()` function, noting that the _SQL return type will be of type JSON_.

In [73]:
function_expr = func.json_object("{a, 1, b, 'def', c, 3.5", type_=JSON)

In [74]:
stmt = select(function_expr["def"])
print(stmt)

SELECT json_object(:json_object_1)[:json_object_2] AS anon_1


##### Built-in Functions Have Pre-Configured Return Types

For _common aggregate functions_ like `count`, `max`, `min` as well as a very small number of _date functions_ like `now` and _string functions_ like `concat`, the `SQL return type` is __set up appropriately__, sometimes based on usage. The `max` function and _similar aggregate filtering functions_ will set up the `SQL return type` __based on the argument given__.

In [75]:
m1 = func.max(Column("some_int", Integer))
m1.type

Integer()

In [76]:
m2 = func.max(Column("some_str", String))
m2.type

String()

`Date and time functions` typically correspond to SQL expressions described by `DateTime`, `Date` or `Time`.

In [77]:
func.now().type

DateTime()

In [78]:
func.current_date().type

Date()

A _known string function_ such as `concat` will know that a SQL expression would be of type `String`.

In [79]:
func.concat("x", "y").type

String()

However, for the _vast majority of SQL functions_, `SQLAlchemy` __does not have them explicitly present__ in its very small list of known functions. For example, while there is typically no issue using SQL functions `func.lower()` and `func.upper()` to __convert the casing of strings__, `SQLAlchemy` _doesn't actually know about these functions_, so they have a `"null"` SQL return type.

In [80]:
func.upper("lowercase").type

NullType()

For _simple functions_ like `upper` and `lower`, the issue is __not usually significant__, as _string values may be received from the database without any special type handling_ on the SQLAlchemy side, and SQLAlchemy's `type coercion rules` can __often correctly guess__ intent as well; the Python `+` operator for example will be __correctly interpreted__ as the `string concatenation operator` based on looking at both sides of the expression.

In [81]:
print(select(func.upper("lowercase") + " suffix"))

SELECT upper(:upper_1) || :upper_2 AS anon_1


Overall, the scenario where the `Function.type_` parameter __is likely necessary__ is:

1. the function is __not already a SQLAlchemy `built-in` function__; this can be _evidenced_ by `creating the function` and _observing the Function.type attribute_.

In [82]:
func.count().type

Integer()

In [83]:
func.json_object("{'a', 'b'}").type

NullType()

2. __`Function-aware` expression support is needed__; this most _typically_ refers to `special operators` _related to datatypes_ such as `JSON` or `ARRAY`.

3. __Result value processing is needed__, which may include types such as `DateTime`, `Boolean`, `Enum`, or again _special datatypes_ such as `JSON`, `ARRAY`.

##### Advanced SQL Function Techniques

The following subsections illustrate more things that can be done with `SQL functions`. While these techniques are `less common` and `more advanced` than basic SQL function use, they nonetheless are __extremely popular__, largely as a _result of PostgreSQL's emphasis on more complex function forms_, including `table- and column-valued forms` that are __popular with JSON data__.

##### Using Window Functions

A `window function` is a _special use_ of a `SQL aggregate function` which _calculates the aggregate value over the rows being returned in a group as the individual result rows are processed_. Whereas a function like `MAX()` will give you the _highest value of a column within a set of rows_, using the `same function as a "window function"` will given you the _highest value for each row, as of that row_.

In SQL, `window functions` allow one to __specify the rows over which the function should be applied__, a `"partition" value` which _considers the window over different sub-sets of rows_, and an `order by` expression which _importantly indicates the order in which rows should be applied to the aggregate function_.

In SQLAlchemy, __all SQL functions generated by the func namespace__ include a method `FunctionElement.over()` which __grants the window function, or `"OVER"`, syntax__; the construct produced is the `Over` construct.

A common function used with window functions is the `row_number()` function which simply __counts rows__. We may _partition_ this `row count` _against user name to number the email addresses of individual users_.

In [84]:
stmt = select(
    func.row_number().over(partition_by=user_table.c.name),
    user_table.c.name,
    address_table.c.email_address,
).select_from(user_table).join(address_table)

with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2022-10-03 17:24:06,350 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:24:06,351 INFO sqlalchemy.engine.Engine SELECT row_number() OVER (PARTITION BY user_account.name) AS anon_1, user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
2022-10-03 17:24:06,352 INFO sqlalchemy.engine.Engine [generated in 0.00202s] ()
[(1, 'sandy', 'sandy@sqlalchemy.org'), (2, 'sandy', 'sandy@squirrelpower.org'), (1, 'spongebob', 'spongebob@sqlalchemy.org')]
2022-10-03 17:24:06,354 INFO sqlalchemy.engine.Engine ROLLBACK


Above, the `FunctionElement.over.partition_by` parameter is used so that the `PARTITION BY` clause is __rendered within the OVER clause__. We also may make use of the `ORDER BY` clause using `FunctionElement.over.order_by`.

In [85]:
stmt = select(
    func.count().over(order_by=user_table.c.name),
    user_table.c.name,
    address_table.c.email_address,
).select_from(user_table).join(address_table)

with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2022-10-03 17:24:06,553 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:24:06,554 INFO sqlalchemy.engine.Engine SELECT count(*) OVER (ORDER BY user_account.name) AS anon_1, user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
2022-10-03 17:24:06,555 INFO sqlalchemy.engine.Engine [generated in 0.00220s] ()
[(2, 'sandy', 'sandy@sqlalchemy.org'), (2, 'sandy', 'sandy@squirrelpower.org'), (3, 'spongebob', 'spongebob@sqlalchemy.org')]
2022-10-03 17:24:06,558 INFO sqlalchemy.engine.Engine ROLLBACK


> ##### Tip
>
> It's important to note that the `FunctionElement.over()` method __only applies__ to those SQL functions which are __in fact aggregate functions__; while the `Over` construct will _happily render itself for any SQL function_ given, the __database will reject the expression if the function itself is not a SQL aggregate function__.

##### Special Modifiers `WITHIN GROUP`, `FILTER`

The `"WITHIN GROUP"` SQL syntax is used in conjunction with an `"ordered set"` or a `"hypothetical set"` aggregate function. Common `"ordered set"` functions include `percentile_cont()` and `rank()`. SQLAlchemy includes __built-in implementations__ `rank`, `dense_rank`, `mode`, `percentile_cont` and `percentile_disc` which include a `FunctionElement.within_group()` method.

In [86]:
print(func.unnest(func.percentile_disc([0.25,0.5,0.75,1]).within_group(user_table.c.name)))

unnest(percentile_disc(:percentile_disc_1) WITHIN GROUP (ORDER BY user_account.name))


`"FILTER"` is _supported by some backends_ to __`limit the range of an aggregate function` to a particular subset of rows compared to the total range of rows returned__, available using the `FunctionElement.filter()` method.

In [87]:
stmt = select(
    func.count(address_table.c.email_address).filter(user_table.c.name == "sandy"),
    func.count(address_table.c.email_address).filter(user_table.c.name == "spongebob"),
).select_from(user_table).join(address_table)

with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2022-10-03 17:24:06,889 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:24:06,890 INFO sqlalchemy.engine.Engine SELECT count(address.email_address) FILTER (WHERE user_account.name = ?) AS anon_1, count(address.email_address) FILTER (WHERE user_account.name = ?) AS anon_2 
FROM user_account JOIN address ON user_account.id = address.user_id
2022-10-03 17:24:06,892 INFO sqlalchemy.engine.Engine [generated in 0.00272s] ('sandy', 'spongebob')
[(2, 1)]
2022-10-03 17:24:06,895 INFO sqlalchemy.engine.Engine ROLLBACK


##### Table-Valued Functions

`Table-valued SQL functions` support a __scalar representation that contains named sub-elements__. Often used for _JSON and ARRAY-oriented functions_ as well as functions like `generate_series()`, the _table-valued function_ is specified in the `FROM` clause, and is then __referred towards as a table, or sometimes even as a column__. Functions of this form are _prominent_ within the `PostgreSQL` database, however some forms of _table-valued functions_ are also supported by `SQLite`, `Oracle`, and `SQL Server`.

SQLAlchemy provides the `FunctionElement.table_valued()` method as the basic `"table-valued function"` construct, which will __convert a func object into a FROM clause containing a series of named columns__, _based on string names passed positionally_. This returns a `TableValuedAlias` object, which is a _function-enabled_ `Alias` construct that may be used as any other `FROM` clause as introduced at `Using Aliases`. Below we illustrate the `json_each()` function, which while common on `PostgreSQL` is also supported by modern versions of `SQLite`.

In [88]:
onetwothree = func.json_each('["one", "two", "three"]').table_valued("value")
stmt = select(onetwothree).where(onetwothree.c.value.in_(["two", "three"]))

with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2022-10-03 17:24:07,105 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:24:07,106 INFO sqlalchemy.engine.Engine SELECT anon_1.value 
FROM json_each(?) AS anon_1 
WHERE anon_1.value IN (?, ?)
2022-10-03 17:24:07,107 INFO sqlalchemy.engine.Engine [generated in 0.00296s] ('["one", "two", "three"]', 'two', 'three')
[('two',), ('three',)]
2022-10-03 17:24:07,110 INFO sqlalchemy.engine.Engine ROLLBACK


Above, we used the `json_each()` __JSON function__ supported by `SQLite` and `PostgreSQL` to _generate a table valued expression with a single column referred towards as value_, and then _selected two of its three rows_.

##### Column Valued Functions - Table Valued Function as a Scalar Column

_A special syntax supported by_ `PostgreSQL` and `Oracle` is that of __referring towards a function__ in the `FROM` clause, which then __delivers itself as a single column__ in the _columns clause_ of a `SELECT` statement or _other column expression context_. `PostgreSQL` makes great use of this syntax for such functions as `json_array_elements()`, `json_object_keys()`, `json_each_text()`, `json_each()`, etc. SQLAlchemy refers to this as a `"column valued" function` and is _available_ by applying the `FunctionElement.column_valued()` modifier to a `Function` construct.

In [89]:
stmt = select(func.json_array_elements('["one", "two"]').column_valued("x"))
print(stmt)

SELECT x 
FROM json_array_elements(:json_array_elements_1) AS x


The `"column valued"` form is _also supported_ by the `Oracle` dialect, where it is __usable for custom SQL functions__.

In [90]:
stmt = select(func.scalar_strings(5).column_valued("s"))
print(stmt.compile(dialect=oracle.dialect()))

SELECT COLUMN_VALUE s 
FROM TABLE (scalar_strings(:scalar_strings_1)) s


#### Data Casts and Type Coercion

In SQL, we often need to _indicate the datatype_ of an expression __explicitly__, either to tell the database _what type is expected_ in an otherwise ambiguous expression, or in some cases when we want to _convert the implied datatype_ of a SQL expression into something else. The SQL `CAST` keyword is used for this task, which in SQLAlchemy is provided by the `cast()` function. This function __accepts a column expression__ and a __data type object__ as arguments, as demonstrated below where we produce a SQL expression `CAST(user_account.id AS VARCHAR)` from the `user_table.c.id` column object.

In [91]:
stmt = select(cast(user_table.c.id, String))
with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2022-10-03 17:24:07,635 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-03 17:24:07,637 INFO sqlalchemy.engine.Engine SELECT CAST(user_account.id AS VARCHAR) AS id 
FROM user_account
2022-10-03 17:24:07,638 INFO sqlalchemy.engine.Engine [generated in 0.00307s] ()
[('1',), ('2',), ('3',)]
2022-10-03 17:24:07,641 INFO sqlalchemy.engine.Engine ROLLBACK


The `cast()` function not only __renders the SQL CAST syntax__, it also _produces a SQLAlchemy column expression_ that will __act as the given datatype on the Python side__ as well. A `string` expression that is `cast()` to `JSON` will gain __JSON subscript and comparison operators__, for example.

In [92]:
print(cast("{'a': 'b'}", JSON)["a"])

CAST(:param_1 AS JSON)[:param_2]
